In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import os
import shutil
from sklearn.preprocessing import MultiLabelBinarizer
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms,models
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from PIL import Image
import ast


Mounted at /content/drive


In [ ]:
one_hot_train_df = pd.read_csv('/content/drive/MyDrive/Deep Final Work/Training dfs/one_hot_train_df.csv')
one_hot_test_df = pd.read_csv('/content/drive/MyDrive/Deep Final Work/Training dfs/one_hot_test_df.csv')

index_train_df = pd.read_csv('/content/drive/MyDrive/Deep Final Work/Training dfs/index_train_df.csv')
index_test_df = pd.read_csv('/content/drive/MyDrive/Deep Final Work/Training dfs/index_test_df.csv')

w2v_train_df = pd.read_csv('/content/drive/MyDrive/Deep Final Work/Training dfs/w2v_train_df.csv')
w2v_test_df = pd.read_csv('/content/drive/MyDrive/Deep Final Work/Training dfs/w2v_test_df.csv')

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),  # Standard ImageNet normalization
])

#Train and evaluate resnet over one-hot

In [ ]:
class CalorieOneHotDataset(Dataset):
    def __init__(self, df, calorie_col='dish_calories', transform=None):

        self.df = df.reset_index(drop=True)
        self.transform = transform
        self.calorie_col = calorie_col

        self.ingredient_cols = [col for col in df.columns if col not in ['path', calorie_col]]

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image_path = row['path']

        image = Image.open(image_path).convert('RGB')
        if self.transform:
            image = self.transform(image)

        # Get one-hot encoded ingredient vector as a tensor
        ingredients_vector = row[self.ingredient_cols].values.astype(np.float32)
        ingredients_vector = torch.tensor(ingredients_vector, dtype=torch.float)

        # Get calorie target
        calories = torch.tensor(row[self.calorie_col], dtype=torch.float)

        return image, ingredients_vector, calories

train_dataset_onehot = CalorieOneHotDataset(one_hot_train_df, transform=transform)
test_dataset_onehot = CalorieOneHotDataset(one_hot_test_df, transform=transform)

train_loader_onehot = DataLoader(train_dataset_onehot, batch_size=8, shuffle=True)
test_loader_onehot = DataLoader(test_dataset_onehot, batch_size=8, shuffle=False)


In [ ]:
class CaloriePredictorOneHot(nn.Module):
    def __init__(self, ingredient_dim, hidden_dim=128):

        super().__init__()
        # Load a pretrained ResNet-18 and remove its classification head.
        self.resnet = models.resnet18(pretrained=True)
        num_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Identity()

        # Process the one-hot ingredient vector.
        self.ingredient_fc = nn.Sequential(
            nn.Linear(ingredient_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU()
        )
        combined_dim = num_features + 32
        self.fc = nn.Sequential(
            nn.Linear(combined_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )

    def forward(self, image, ingredient_vector):
        img_feat = self.resnet(image)
        ingr_feat = self.ingredient_fc(ingredient_vector)
        combined = torch.cat([img_feat, ingr_feat], dim=1)
        out = self.fc(combined)
        return out.squeeze(1)


In [ ]:

# -------------------------------
# Training and Evaluation Code
# -------------------------------

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

ingredient_dim = len(train_dataset_onehot.ingredient_cols)

onehot_model = CaloriePredictorOneHot(ingredient_dim=ingredient_dim).to(device)

criterion = nn.MSELoss()  # Mean Squared Error for regression.
optimizer = optim.Adam(onehot_model.parameters(), lr=1e-4)

num_epochs = 20

for epoch in range(num_epochs):
    # Training phase.
    onehot_model.train()
    total_train_loss = 0.0
    for images, ingredients_vector, calories in train_loader_onehot:
        images = images.to(device)
        ingredients_vector = ingredients_vector.to(device)
        calories = calories.to(device)

        optimizer.zero_grad()
        outputs = onehot_model(images, ingredients_vector)
        loss = criterion(outputs, calories)
        loss.backward()
        optimizer.step()

        total_train_loss += loss.item() * images.size(0)
    avg_train_loss = total_train_loss / len(train_dataset_onehot)

    # Evaluation phase.
    onehot_model.eval()
    total_test_loss = 0.0
    with torch.no_grad():
        for images, ingredients_vector, calories in test_loader_onehot:
            images = images.to(device)
            ingredients_vector = ingredients_vector.to(device)
            calories = calories.to(device)

            outputs = onehot_model(images, ingredients_vector)
            loss = criterion(outputs, calories)
            total_test_loss += loss.item() * images.size(0)
    avg_test_loss = total_test_loss / len(test_dataset_onehot)

    print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {avg_train_loss:.4f} | Test Loss: {avg_test_loss:.4f}")

# Save the trained model.
torch.save(onehot_model,"/content/drive/MyDrive/Deep Final Work/Saved Models/calorie_predictor_resnet_onehot1.pth")
print("✅ Model training complete and saved!")

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 104MB/s]


Epoch 1/20 | Train Loss: 65611.3316 | Test Loss: 17340.3350
Epoch 2/20 | Train Loss: 21335.1766 | Test Loss: 7255.3729
Epoch 3/20 | Train Loss: 15157.8153 | Test Loss: 10798.5745
Epoch 4/20 | Train Loss: 12617.5136 | Test Loss: 9053.1739
Epoch 5/20 | Train Loss: 10070.1696 | Test Loss: 5600.4849
Epoch 6/20 | Train Loss: 8705.5140 | Test Loss: 5883.8551
Epoch 7/20 | Train Loss: 6925.2571 | Test Loss: 7002.7781
Epoch 8/20 | Train Loss: 6198.0118 | Test Loss: 5439.5540
Epoch 9/20 | Train Loss: 5468.4597 | Test Loss: 5520.9571
Epoch 10/20 | Train Loss: 4843.2717 | Test Loss: 4872.8347
Epoch 11/20 | Train Loss: 4539.3202 | Test Loss: 5751.6928
Epoch 12/20 | Train Loss: 4284.4866 | Test Loss: 4422.0901
Epoch 13/20 | Train Loss: 4072.8774 | Test Loss: 5254.1906
Epoch 14/20 | Train Loss: 4187.6054 | Test Loss: 6938.7376
Epoch 15/20 | Train Loss: 3849.8642 | Test Loss: 5493.1738
Epoch 16/20 | Train Loss: 3575.3023 | Test Loss: 5158.2928
Epoch 17/20 | Train Loss: 3470.4702 | Test Loss: 4723.6907

# Train and evaluate resnet over index Embedding

In [ ]:
import pickle

# Load dictionary from pickle file
with open("/content/drive/MyDrive/Deep Final Work/Training dfs/ingredient_to_index.pkl", "rb") as f:
    ingredient_to_index = pickle.load(f)

print(ingredient_to_index)

{'almonds': 1, 'apple': 2, 'artichokes': 3, 'arugula': 4, 'asparagus': 5, 'avocado': 6, 'baby carrots': 7, 'bacon': 8, 'bagels': 9, 'banana with peel': 10, 'basil': 11, 'beef': 12, 'beets': 13, 'bell peppers': 14, 'berries': 15, 'black beans': 16, 'blackberries': 17, 'blue cheese': 18, 'blueberries': 19, 'bok choy': 20, 'bread': 21, 'broccoli': 22, 'brown rice': 23, 'brown sugar': 24, 'brownies': 25, 'brussels sprouts': 26, 'bulgur': 27, 'butter': 28, 'buttermilk': 29, 'cabbage': 30, 'caesar dressing': 31, 'caesar salad': 32, 'cantaloupe': 33, 'carrot': 34, 'cauliflower': 35, 'celery': 36, 'celery root': 37, 'cereal': 38, 'chard': 39, 'chayote squash': 40, 'cheese': 41, 'cheese pizza': 42, 'cherry tomatoes': 43, 'chia seeds': 44, 'chicken': 45, 'chicken apple sausage': 46, 'chicken breast': 47, 'chicken salad': 48, 'chicken thighs': 49, 'chickpeas': 50, 'chilaquiles': 51, 'chili': 52, 'chive': 53, 'cilantro': 54, 'cod': 55, 'cookies': 56, 'corn': 57, 'corn on the cob': 58, 'cottage che

In [ ]:
class CalorieIndexDataset(Dataset):
    def __init__(self, df, transform=None, calorie_col='dish_calories', ingredient_col='ingredients_embedding'):
        """
        Args:
            df (pd.DataFrame): DataFrame with columns:
                - 'path': image file path.
                - ingredient_col: padded list of ingredient indices (or its string representation).
                - calorie_col: dish calorie value.
            transform: Image transformations.
        """
        self.df = df.reset_index(drop=True)
        self.transform = transform
        self.calorie_col = calorie_col
        self.ingredient_col = ingredient_col

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image = Image.open(row['path']).convert('RGB')
        if self.transform:
            image = self.transform(image)
        ingredient_data = row[self.ingredient_col]
        # If ingredient_data is a string, convert it to a list.
        if isinstance(ingredient_data, str):
            ingredient_data = ast.literal_eval(ingredient_data)
        ingredient_indices = torch.tensor(ingredient_data, dtype=torch.long)
        # Get calorie target.
        calories = torch.tensor(row[self.calorie_col], dtype=torch.float)
        return image, ingredient_indices, calories


train_dataset_index = CalorieIndexDataset(index_train_df, transform=transform)
test_dataset_index  = CalorieIndexDataset(index_test_df, transform=transform)

train_loader_index = DataLoader(train_dataset_index, batch_size=8, shuffle=True)
test_loader_index  = DataLoader(test_dataset_index, batch_size=8, shuffle=False)


In [ ]:
class CaloriePredictorIndices(nn.Module):
    def __init__(self, vocab_size, embed_dim=50, hidden_dim=128):
        """
        Args:
            vocab_size (int): Number of unique ingredients.
            embed_dim (int): Embedding dimension.
            hidden_dim (int): Hidden units in the FC layer after concatenation.
        """
        super().__init__()
        self.resnet = models.resnet18(pretrained=True)
        num_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Identity()
        self.embedding = nn.Embedding(num_embeddings=vocab_size + 1, embedding_dim=embed_dim, padding_idx=0)
        combined_dim = num_features + embed_dim
        self.fc = nn.Sequential(
            nn.Linear(combined_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)  # Regression output for calories.
        )

    def forward(self, image, ingredient_indices):
        image_features = self.resnet(image)
        ingr_embeds = self.embedding(ingredient_indices)
        ingredient_features = ingr_embeds.mean(dim=1)
        combined = torch.cat([image_features, ingredient_features], dim=1)
        out = self.fc(combined)
        return out.squeeze(1)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

vocab_size = len(ingredient_to_index)
index_model = CaloriePredictorIndices(vocab_size=vocab_size, embed_dim=50, hidden_dim=128)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
index_model = index_model.to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(index_model.parameters(), lr=1e-4)
num_epochs = 10

for epoch in range(num_epochs):
    index_model.train()
    total_train_loss = 0.0
    for images, ingredient_indices, calories in train_loader_index:
        images = images.to(device)
        ingredient_indices = ingredient_indices.to(device)
        calories = calories.to(device)

        optimizer.zero_grad()
        outputs = index_model(images, ingredient_indices)
        loss = criterion(outputs, calories)
        loss.backward()
        optimizer.step()

        total_train_loss += loss.item() * images.size(0)
    avg_train_loss = total_train_loss / len(train_dataset_index)

    index_model.eval()
    total_test_loss = 0.0
    with torch.no_grad():
        for images, ingredient_indices, calories in test_loader_index:
            images = images.to(device)
            ingredient_indices = ingredient_indices.to(device)
            calories = calories.to(device)

            outputs = index_model(images, ingredient_indices)
            loss = criterion(outputs, calories)
            total_test_loss += loss.item() * images.size(0)
    avg_test_loss = total_test_loss / len(test_dataset_index)

    print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {avg_train_loss:.4f} | Test Loss: {avg_test_loss:.4f}")

# Save the trained model.
torch.save(index_model, "/content/drive/MyDrive/Deep Final Work/Saved Models/calorie_predictor_resnet_indices.pth")
print("✅ Model training complete and saved!")

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/10 | Train Loss: 65632.9159 | Test Loss: 33841.0128
Epoch 2/10 | Train Loss: 22454.1116 | Test Loss: 7714.6401
Epoch 3/10 | Train Loss: 15666.6690 | Test Loss: 8057.5698
Epoch 4/10 | Train Loss: 13560.1747 | Test Loss: 14052.9516
Epoch 5/10 | Train Loss: 10205.4769 | Test Loss: 8512.7222
Epoch 6/10 | Train Loss: 8162.1674 | Test Loss: 10292.6881
Epoch 7/10 | Train Loss: 7060.4899 | Test Loss: 9101.5987
Epoch 8/10 | Train Loss: 5511.3396 | Test Loss: 7238.1546
Epoch 9/10 | Train Loss: 5886.6755 | Test Loss: 8765.4848
Epoch 10/10 | Train Loss: 5035.7313 | Test Loss: 9943.3935
✅ Model training complete and saved!


Training and evaluating word2vec model

In [ ]:
class CalorieDatasetWithEmbeddings(Dataset):
    def __init__(self, df, transform=None, calorie_col='dish_calories'):
        """
        Args:
            df (pd.DataFrame): DataFrame with 'path', 'ingredient_embedding', and calorie info.
            transform: Image transformations to apply.
            calorie_col (str): Column name for the calorie target.
        """
        self.df = df.reset_index(drop=True)
        self.transform = transform
        self.calorie_col = calorie_col

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image = Image.open(row['path']).convert('RGB')
        if self.transform:
            image = self.transform(image)

        embedding_data = row['ingredient_embedding']
        if isinstance(embedding_data, str):
            # If the string contains commas, use literal_eval
            if ',' in embedding_data:
                embedding_data = ast.literal_eval(embedding_data)
            else:
                # Otherwise, assume it's space-separated.
                embedding_data = embedding_data.strip("[]")
                # Split on whitespace and convert each element to float.
                embedding_data = [float(x) for x in embedding_data.split()]
        ingredient_embedding = torch.tensor(embedding_data, dtype=torch.float)

        calories = torch.tensor(row[self.calorie_col], dtype=torch.float)

        return image, ingredient_embedding, calories


train_dataset_w2v = CalorieDatasetWithEmbeddings(w2v_train_df, transform=transform)
test_dataset_w2v  = CalorieDatasetWithEmbeddings(w2v_test_df, transform=transform)

train_loader_w2v = DataLoader(train_dataset_w2v, batch_size=8, shuffle=True)
test_loader_w2v  = DataLoader(test_dataset_w2v, batch_size=8, shuffle=False)


In [ ]:
class CaloriePredictorW2V(nn.Module):
    def __init__(self, input_embed_dim=50, hidden_dim=128):
        """
        Args:
            input_embed_dim (int): Dimension of the precomputed ingredient embedding.
            hidden_dim (int): Number of hidden units in the FC regressor.
        """
        super().__init__()
        self.resnet = models.resnet18(pretrained=True)
        num_img_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Identity()
        combined_dim = num_img_features + input_embed_dim
        self.fc = nn.Sequential(
            nn.Linear(combined_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )

    def forward(self, image, ingredient_embedding):
        image_features = self.resnet(image)
        combined = torch.cat([image_features, ingredient_embedding], dim=1)
        out = self.fc(combined)
        return out.squeeze(1)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
w2v_model = CaloriePredictorW2V(input_embed_dim=50, hidden_dim=128).to(device)

criterion = nn.MSELoss()  # Mean Squared Error for regression.
optimizer = optim.Adam(w2v_model.parameters(), lr=1e-4)
num_epochs = 20

for epoch in range(num_epochs):
    w2v_model.train()
    total_train_loss = 0.0
    for images, ingredient_embedding, calories in train_loader_w2v:
        images = images.to(device)
        ingredient_embedding = ingredient_embedding.to(device)
        calories = calories.to(device)

        optimizer.zero_grad()
        outputs = w2v_model(images, ingredient_embedding)
        loss = criterion(outputs, calories)
        loss.backward()
        optimizer.step()

        total_train_loss += loss.item() * images.size(0)
    avg_train_loss = total_train_loss / len(train_dataset_w2v)

    w2v_model.eval()
    total_test_loss = 0.0
    with torch.no_grad():
        for images, ingredient_embedding, calories in test_loader_w2v:
            images = images.to(device)
            ingredient_embedding = ingredient_embedding.to(device)
            calories = calories.to(device)

            outputs = w2v_model(images, ingredient_embedding)
            loss = criterion(outputs, calories)
            total_test_loss += loss.item() * images.size(0)
    avg_test_loss = total_test_loss / len(test_dataset_w2v)

    print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {avg_train_loss:.4f} | Test Loss: {avg_test_loss:.4f}")

# Save the trained model.
torch.save(w2v_model, "/content/drive/MyDrive/Deep Final Work/Saved Models/calorie_predictor_resnet_word2vec1.pth")
print("✅ Model training complete and saved!")

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/20 | Train Loss: 68311.9144 | Test Loss: 17228.9120
Epoch 2/20 | Train Loss: 23529.6648 | Test Loss: 9601.0068
Epoch 3/20 | Train Loss: 16174.5292 | Test Loss: 6315.2299
Epoch 4/20 | Train Loss: 13891.9486 | Test Loss: 7445.8075
Epoch 5/20 | Train Loss: 11062.4659 | Test Loss: 6077.9290
Epoch 6/20 | Train Loss: 8528.8354 | Test Loss: 6233.7331
Epoch 7/20 | Train Loss: 6702.3585 | Test Loss: 7106.9502
Epoch 8/20 | Train Loss: 6025.6422 | Test Loss: 5890.2245
Epoch 9/20 | Train Loss: 5131.3018 | Test Loss: 5549.0421
Epoch 10/20 | Train Loss: 4978.7654 | Test Loss: 8483.1475
Epoch 11/20 | Train Loss: 4150.4109 | Test Loss: 7399.6059
Epoch 12/20 | Train Loss: 3994.0604 | Test Loss: 4903.1848
Epoch 13/20 | Train Loss: 3731.2161 | Test Loss: 5010.8068
Epoch 14/20 | Train Loss: 3409.6811 | Test Loss: 5393.4058
Epoch 15/20 | Train Loss: 3128.4786 | Test Loss: 7174.9706
Epoch 16/20 | Train Loss: 3081.4382 | Test Loss: 7377.7380
Epoch 17/20 | Train Loss: 3139.4959 | Test Loss: 7919.8309
